In [ ]:
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
import torchvision
import timm
from torch import nn, optim
from torch.utils.data import DataLoader
import os
from torchvision import datasets, transforms
from google.colab import drive
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.utils.class_weight import compute_class_weight
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_path = "/content/drive/My Drive/wheat_data.zip"

In [ ]:
extract_path = "/content/wheat_data"

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
      zip_ref.extractall(extract_path)

# Check it's there
print(os.listdir(extract_path))

['wheat_data', '__MACOSX']


^ 3 min to run

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

base_path = "/content/wheat_data/wheat_data"  # Fixed path

train_dir = os.path.join(base_path, "train")
valid_dir = os.path.join(base_path, "valid")
test_dir  = os.path.join(base_path, "test")


# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform)
test_dataset  = datasets.ImageFolder(test_dir,  transform=transform)

# Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
from torchvision import models
import torch.nn as nn

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 52.2MB/s]


In [ ]:
!pip install tqdm
from tqdm.notebook import tqdm

In [ ]:
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Early stopping setup
best_val_loss = float('inf')
patience = 3
counter = 0

In [ ]:
epochs = 20
best_val_loss = float('inf')
patience = 5
counter = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    train_loss /= len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        loop = tqdm(valid_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False)
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            loop.set_postfix(val_loss=loss.item())

    val_loss /= len(valid_loader)

    print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

    # Early stopping logic (no saving)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        print(f"Early stopping patience: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered.")
            break

Epoch 1/20 [Train]:   0%|          | 0/410 [00:00<?, ?it/s]

Epoch 1/20 [Val]:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1 - Train Loss: 0.8355 - Val Loss: 0.8026


Epoch 2/20 [Train]:   0%|          | 0/410 [00:00<?, ?it/s]

Epoch 2/20 [Val]:   0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch 13/20 [Val]:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 13 - Train Loss: 0.0717 - Val Loss: 0.6172
Early stopping patience: 4/5


Epoch 14/20 [Train]:   0%|          | 0/410 [00:00<?, ?it/s]

Epoch 14/20 [Val]:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 14 - Train Loss: 0.0691 - Val Loss: 1.0626
Early stopping patience: 5/5
Early stopping triggered.


# Model evaluation

In [ ]:
model.eval()
# set model to evaluation mode

correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating on Test Set"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Evaluating on Test Set:   0%|          | 0/24 [00:00<?, ?it/s]

Test Accuracy: 92.40%


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [ ]:
from collections import defaultdict

model.eval()

all_preds = []
all_labels = []
class_correct = defaultdict(int)
class_total = defaultdict(int)

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating for Confusion Matrix"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        for label, pred in zip(labels, predicted):
            class_total[label.item()] += 1
            if label == pred:
                class_correct[label.item()] += 1

Evaluating for Confusion Matrix:   0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
report_dict = classification_report(
    all_labels,
    all_preds,
    target_names=train_dataset.classes,
    output_dict=True,
    zero_division=0
)

In [ ]:
# Convert report dict to DataFrame
metrics_df = pd.DataFrame(report_dict).transpose()

# Keep only class rows (filter out 'accuracy', 'macro avg', etc.)
metrics_df = metrics_df.loc[train_dataset.classes]

# Round for readability
metrics_df = metrics_df.round(2)

# Print the DataFrame
print(metrics_df)

                      precision  recall  f1-score  support
Aphid                      1.00    0.96      0.98     50.0
Black Rust                 1.00    0.96      0.98     50.0
Blast                      0.94    1.00      0.97     50.0
Brown Rust                 1.00    1.00      1.00     50.0
Common Root Rot            1.00    1.00      1.00     50.0
Fusarium Head Blight       1.00    0.98      0.99     50.0
Healthy                    1.00    0.10      0.18     50.0
Leaf Blight                0.89    0.96      0.92     50.0
Mildew                     1.00    0.98      0.99     50.0
Mite                       1.00    0.96      0.98     50.0
Septoria                   1.00    1.00      1.00     50.0
Smut                       1.00    1.00      1.00     50.0
Stem fly                   1.00    1.00      1.00     50.0
Tan spot                   0.91    0.96      0.93     50.0
Yellow Rust                0.54    1.00      0.70     50.0


In [ ]:
# Store results in a list of dictionaries
accuracy_data = []

for idx, class_name in enumerate(train_dataset.classes):
    accuracy = 100 * class_correct[idx] / class_total[idx] if class_total[idx] > 0 else 0.0
    accuracy_data.append({
        "Class": class_name,
        "Accuracy (%)": round(accuracy, 2),
        "Correct": class_correct[idx],
        "Total": class_total[idx]
    })

# Convert to DataFrame
accuracy_df = pd.DataFrame(accuracy_data)

# Print the DataFrame
print(accuracy_df)


                   Class  Accuracy (%)  Correct  Total
0                  Aphid          96.0       48     50
1             Black Rust          96.0       48     50
2                  Blast         100.0       50     50
3             Brown Rust         100.0       50     50
4        Common Root Rot         100.0       50     50
5   Fusarium Head Blight          98.0       49     50
6                Healthy          10.0        5     50
7            Leaf Blight          96.0       48     50
8                 Mildew          98.0       49     50
9                   Mite          96.0       48     50
10              Septoria         100.0       50     50
11                  Smut         100.0       50     50
12              Stem fly         100.0       50     50
13              Tan spot          96.0       48     50
14           Yellow Rust         100.0       50     50
